# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [27]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [28]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [29]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [30]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's.

In [31]:
doi_counted = doi.value_counts()
doi_counted

10.1016/j.dsx.2020.04.012       2
10.31729/jnma.5498              2
10.1093/nar/gki110              1
10.1101/2020.02.16.20023671     1
10.3390/cancers12123490         1
                               ..
10.4061/2009/239204             1
10.1007/s00262-010-0903-4       1
10.1007/978-3-030-50423-6_15    1
10.1016/j.eclinm.2020.100512    1
10.7759/cureus.10923            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [32]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [33]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [34]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [35]:
return_example = AffiliationsFromScopusByDOI(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [36]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")

The DataFrame is empty


The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [37]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

0

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [38]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < 5:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 0 -> 10.1016/j.dsx.2020.04.012
Position: 1 -> 10.31729/jnma.5498
Position: 2 -> 10.1093/nar/gki110
Position: 3 -> 10.1101/2020.02.16.20023671
Position: 4 -> 10.3390/cancers12123490
Wall time: 15.5 s


The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [39]:
def save_new_extra_info(len_df_current_extra_info, upto):
    """
    This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
    As a result, this function returns a range of valid entries up to the given parameter. 
    """
    dict_new_extra_info_saver = dict()
    i = len_df_current_extra_info
    while i < upto:
        #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
        dict_new_extra_info_saver[i] = dict_new_extra_info[i]
        i = i + 1 
    return dict_new_extra_info_saver

In [40]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    #Prior appending, the dictionary is transformed to a DataFrame
    df_newly_fetched = pd.DataFrame(dic)
    
    #For readability, the DataFrame is transposed
    df_newly_fetched_transposed = df_newly_fetched.T
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

The existing and newly fetched information are combined into one DataFrame. 

In [41]:
df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

,affiliation,coredata
0,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
1,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
2,"[{'affiliation-city': 'St. Louis', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': 'DAT..."
3,None,None
4,"{'affiliation-city': 'Campinas', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85096696345', ..."


In [42]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object.

In [50]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    return ser_affiliation, ser_coredata

In [51]:
stored_series = store_df_columns(df_combined_extra_info)
stored_series[0]

0    [{'affiliation-city': 'New Delhi', 'affilname'...
1    [{'affiliation-city': None, 'affilname': 'Pata...
2    [{'affiliation-city': 'St. Louis', 'affilname'...
3                                                 None
4    {'affiliation-city': 'Campinas', 'affilname': ...
Name: affiliation, dtype: object

In [52]:
stored_series[1]

0    {'srctype': 'j', 'eid': '2-s2.0-85083171050', ...
1    {'srctype': 'j', 'prism:issueIdentifier': '230...
2    {'srctype': 'j', 'prism:issueIdentifier': 'DAT...
3                                                 None
4    {'srctype': 'j', 'eid': '2-s2.0-85096696345', ...
Name: coredata, dtype: object

The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads. 

In [ ]:
ser_affiliation.to_pickle('extra_info_affiliation_CS5099.pkl')
ser_coredata.to_pickle('extra_info_coredata_CS5099.pkl')

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [ ]:
df_current_extra_info_checker = df_current_extra_info
df_current_extra_info_checker['doi'] = doi_counted.index
df_current_extra_info_checker.head(50)

In [ ]:
len_df_current_extra_info_checker = len(df_current_extra_info_checker)
dict_new_extra_info_checker = dict()
i = 0 
while i < 50:
    if df_current_extra_info_checker['affiliation'][i] == None:
        dict_new_extra_info_checker[i] = AffiliationsFromScopusByDOI(client, df_current_extra_info_checker['doi'][i])
        #print("Position: " + str(i) + " -> " +  df_current_extra_info_checker['doi'][i])
        print(json.dumps(dict_new_extra_info_checker[i], indent=2))
    i = i + 1    

In [ ]:
dict_new_extra_info_checker

In [ ]:
df_new_extra_info_new_found = pd.DataFrame(dict_new_extra_info_checker)
df_new_extra_info_new_found_transposed = df_new_extra_info_new_found.T
df_new_extra_info_new_found_transposed

In [ ]:
test_dataframe = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
test_dataframe.head(50)